## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import timm
from torchvision.models import video

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
class CFG:
    model_name = "mvit_v2_s"
    n_folds = 5
    n_classes = 13
    video_length=50
    img_size=256
    epochs=50
    lr=3e-4
    batch_size=16
    seed=41
        

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed) # Seed 고정

## Data Load

In [5]:
df = pd.read_csv('./train.csv')

## Train / Validation Split

In [6]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG.seed)

## CustomDataset

In [7]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG.video_length):
            _, img = cap.read()
            img = cv2.resize(img, (CFG.img_size, CFG.img_size))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [8]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

In [9]:
#train_dataset[0][0].shape
next(iter(train_loader))[0].shape

torch.Size([16, 3, 50, 256, 256])

## Model Define

In [10]:
#from torchsummary import summary
#
#
#model = BaseModel().to(device)
#summary(model, input_size=(3, 50, 128,128))

In [11]:
from torchvision.models.video import r3d_18

class BaseModel(nn.Module):
    def __init__(self, num_classes=CFG.n_classes, fc_type='shallow'):
        super(BaseModel, self).__init__()
        self.fc_type=fc_type
        self.num_classes=num_classes
        self.backbone = getattr(video,CFG.model_name)(pretrained=True)
        self.fc=self.get_fc()

        
    def get_fc(self):
#        if self.fc_type == 'deep':
#            fc = nn.Sequential(nn.Linear(self.backbone.fc.in_features, self.backbone.fc.in_features//2),
#                                        nn.BatchNorm1d(self.backbone.fc.in_features//2,  momentum=0.1),
#                                        nn.ReLU(),
#                                        nn.Linear(self.backbone.fc.in_features//2, self.backbone.fc.in_features//4),
#                                        nn.BatchNorm1d(self.backbone.fc.in_features//4,  momentum=0.1),
#                                        nn.ReLU(),
#                                        nn.Linear(self.backbone.fc.in_features//4, self.num_classes))
    
        if self.fc_type == 'shallow':
            fc = nn.Linear(400, self.num_classes)
        else:
            raise ValueError(f"Wrong fc-type input {self.fc_type}")
        return fc
    
    def forward(self, x):
        x = self.backbone(x)
        x= self.fc(x)
        return x

In [12]:
model=BaseModel(fc_type='shallow')
model

BaseModel(
  (backbone): MViT(
    (conv_proj): Conv3d(3, 96, kernel_size=(3, 7, 7), stride=(2, 4, 4), padding=(1, 3, 3))
    (pos_encoding): PositionalEncoding()
    (blocks): ModuleList(
      (0): MultiscaleBlock(
        (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
        (attn): MultiscaleAttention(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (project): Sequential(
            (0): Linear(in_features=96, out_features=96, bias=True)
          )
          (pool_q): Pool(
            (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=96, bias=False)
            (norm_act): Sequential(
              (0): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            )
          )
          (pool_k): Pool(
            (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 8, 8), padding=(1, 1, 1), groups=96, bias=False)
  

## Train

In [13]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG.epochs+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

## Run!!

In [15]:
model = BaseModel(fc_type='shallow')
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG.lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.00001)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/135 [00:00<?, ?it/s]

RuntimeError: shape '[16, 96, 8, 56, 56]' is invalid for input of size 157286400

## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
preds = inference(model, test_loader, device)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['label'] = preds
submit.head()

In [ ]:
submit.to_csv('./mvitv2.csv', index=False)